<a href="https://colab.research.google.com/github/imabari/ehime-covid-bot/blob/main/ehime_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インストール

In [1]:
!pip install pdfminer.six

     |████████████████████████████████| 5.6MB 6.6MB/s 
     |████████████████████████████████| 2.6MB 44.8MB/s 


# プログラム

## 設定

In [2]:
import datetime
import pathlib
import re
import unicodedata
from urllib.parse import urljoin

In [3]:
from pdfminer.high_level import extract_text
import requests
import tweepy
from bs4 import BeautifulSoup

In [4]:
# 愛媛県の新型コロナウイルス感染症に関する情報
url = "https://www.pref.ehime.jp/h25500/kansen/covid19.html"

## 関数

In [5]:
# スクレイピング
def fetch_soup(url, parser="html.parser"):

    r = requests.get(url)
    r.raise_for_status()

    soup = BeautifulSoup(r.content, parser)

    return soup

In [6]:
# PDFファイルからテキスト抽出
def fetch_text(url):

    r = requests.get(url)

    p = pathlib.Path(pathlib.PurePath(url).name)

    with p.open(mode="wb") as fw:
        fw.write(r.content)

    text = extract_text(p)

    return unicodedata.normalize("NFKC", text)

## スクレイピング

In [7]:
soup = fetch_soup(url)

In [8]:
# 記事のリンクを抽出
tags = soup.select("div#tmp_contents > ul > li > a")

## 新型コロナウイルスの感染の確認等について

In [9]:
# 最新記事は上から4番目
tag1 = tags[3]

In [10]:
# タイトル確認
title1 = tag1.get_text(strip=True)
print(title1)

（1月22日公表分）新型コロナウイルスの感染の確認等について（173・182・271・274・281・285・286事例目関連及び新規9事例）（PDF：141KB）


In [11]:
# PDFのURLを作成
link1 = urljoin(url, tag1.get("href"))

In [12]:
# PDFファイルをダウンロード
text1 = fetch_text(link1)

In [13]:
# テキスト確認
print(text1)

愛媛県保健福祉部健康衛生局 
健康増進課(内線 2402) 

令和3年1月 22 日 
 

 

 
 

 

 

新型コロナウイルスの感染の確認について 

昨日(1月 21 日)、県内で新型コロナウイルスの陽性者が 18 名確認されました。 
積極的疫学調査及び健康観察等を実施してまいります。 

 
○既存・新規人数 
・濃厚接触者及び接触者の検査により判明した陽性者数:9名 
・上記以外の検査により新たに判明した陽性者数      :9名 
 
○陽性者の概要 

陽性者 

年代 

性別 

居住地 

職業 

892 人目 

~
909 人目 

 

10 代    :1名 
20 代    :5名 
30 代    :3名 
40 代    :1名 
50 代    :2名 
60 代    :3名 
70 代    :2名 
80 代    :1名 

男性:6名 

 

女性:12 名 

松山市    :12 名 
今治市    :1名 
西条市    :1名 
四国中央市:3名 
砥部町    :1名 

公務員    :3名 
会社員    :2名 
自営業    :1名 
福祉関係  :1名 
学校関係  :2名 
団体役員  :1名 
団体職員  :2名 
アルバイト:3名 
無  職    :3名 

 
○症状の有無                               ○感染経路等 

       16 名 ※全て軽症 

  家庭内 

あり 

なし 
 

 
 

2名 
 

  同僚・職場内 

  医療機関・社会福祉施設等内 
  県外 
  調査中 
 

 
※現段階の調査で感染経路の可能性が推測される 

:3名 

:4名 

:2名 
:1名 
:8名 
 

 

   

 

 

 
 
○クラスターの状況(陽性者の概要等は上記に含まれる) 

場面等 

事例 

新たな陽性者 

陽性者の状況 

173 事例目  警察署  1名 

職場内クラスター3 
累計陽性者 17 名(警察署 13 名(+1)、警察署以外の者 4 名) 

271 事例目 

職員1名 
入所者1名 
施設以外の者1名 
                         
 

高齢者施設クラス

# 出力

### 全体

In [14]:
m1 = re.search("昨日\((.+)\)、県内で新型コロナウイルスの陽性者が(.+)名確認されました。", text1)
s1 = m1.group(0).replace(" ", "")
print(s1)

昨日(1月21日)、県内で新型コロナウイルスの陽性者が18名確認されました。


### 既存事例

In [15]:
m2 = re.search("濃厚接触者及び接触者の検査により判明した陽性者数 *?:(\d+)名", text1)
s2 = m2.group(0).replace(" ", "")
print(s2)

濃厚接触者及び接触者の検査により判明した陽性者数:9名


### 新規事例

In [16]:
m3 = re.search("上記以外の検査により新たに判明した陽性者数 *?:(\d+)名", text1)
s3 = m3.group(0).replace(" ", "")
print(s3)

上記以外の検査により新たに判明した陽性者数:9名


### 文章結合

In [17]:
print(f"{s1}\n{s2}\n{s3}")

昨日(1月21日)、県内で新型コロナウイルスの陽性者が18名確認されました。
濃厚接触者及び接触者の検査により判明した陽性者数:9名
上記以外の検査により新たに判明した陽性者数:9名


# 愛媛県内の状況

In [18]:
# 記事は上から3番目
tag2 = tags[2]

In [19]:
# タイトル確認
title2 = tag2.get_text(strip=True)
print(title2)

（1月22日9時現在）愛媛県内の状況（PDF：1,161KB）


In [20]:
# PDFのURLを作成
link2 = urljoin(url, tag2.get("href"))

In [21]:
# PDFファイルをダウンロード
text2 = fetch_text(link2)

In [22]:
# テキスト確認
print(text2)

愛 媛 県 内 の 状 況

【 R3.1.22 9時現在 】

事 例

公表日 検査数

陰 性

陽 性

関係者
調査

PCR
検査

健康
観察

4,935

4,420

515

<封じ込め事例>

対処事例1:186事例
※193、252、262:欠番

松山市保健所:4事例
(117、180、212、264事例目)

西条保健所:2事例
(187、188事例目)

八幡浜保健所:1事例
(174事例目)

宇和島保健所:1事例
(185事例目)

<囲い込み事例>

対処事例2
(136、156-2、160、163、
169、178、186、189、
192、194、196~198-1、
199~209、211、213~218、
221~223、225~228、
230、231、233、234、
236~242、244、246、
249~251、254、255、
257事例目)

198-2事例目
【高齢者施設4】
256事例目
(中予保健所)
松山市保健所:6事例
(232、259、266、267、
270、273事例目)

西条保健所:1事例
(248事例目)

中予保健所:2事例
(125、268事例目)

<調査中事例>

対処事例3
(220、235、253、260、
263、265、269、272、
275、279、280、283、
287事例目)

1/16

1/17

173事例目
【今治警察署】
271事例目
【高齢者施設5】
182事例目
(松山市保健所)

1/7

1/19

1/7

1,485

1,309

176

●

●

○

81

9

44

12

62

7

38

11

(1)
42
(1)
86

105

10

445

(37)
220
(10)
79
(40)
53

(1)
36
(1)
81

88

8

417

(36)
203
(7)
62
(39)
48

214

182

●

●

●

●

●

●

●

●

●

●

〇

〇

〇

〇

19

2

6

1

6

5

2

17

28

32

(1)
17
(3)
17
(1)
5

●

●

●

●

●

●

●

●

●

●

〇

〇

〇

〇

●

●

●


In [23]:
m4 = re.search("^新規事例\n(\d+)事例合計", text2, re.MULTILINE | re.DOTALL)
s4 = m4.group(0).replace(" ", "")
print(s4)

新規事例
9事例合計


# 抽出

In [24]:
result = {}

In [25]:
result["日付"] = m1.group(1).replace(" ", "")
result["全体人数"] = m1.group(2).replace(" ", "")
result["既存人数"] = m2.group(1)
result["新規人数"] = m3.group(1)
result["新規事例"] = m4.group(1)

In [26]:
print(result)

{'日付': '1月21日', '全体人数': '18', '既存人数': '9', '新規人数': '9', '新規事例': '9'}


In [27]:
twit = f'{result["日付"]}\n全体{result["全体人数"]}名\n既存事例{result["既存人数"]}名\n新規{result["新規事例"]}事例{result["新規人数"]}名'
print(twit)

1月21日
全体18名
既存事例9名
新規9事例9名
